# Configuracion inicial

In [ ]:
!pip install langchain
!pip install openai

# Introduccion 

En este caso trabajaremos con dos tipos de memoria que son los más comunes y usadas


* [``ConversationBufferMemory``](https://python.langchain.com/docs/modules/memory/types/buffer): Almacena los mensajes de chat en una lista en memoria. Puede devolver los mensajes como una cadena concatenada o como una lista de objetos ``ChatMessage``.

* [``Conversation Knowledge Graph``](https://python.langchain.com/docs/modules/memory/types/kg):  Este tipo de memoria utiliza grafos para recrear una memoria.

# ConversationBufferMemory

En este ejemplo, creamos una instancia de ConversationBufferMemory y luego agregamos mensajes de usuario y de IA a la memoria utilizando los métodos add_user_message y add_ai_message. Finalmente, utilizamos el método get_chat_history para obtener el historial de chat almacenado en la memoria.

## Creacion de la memoria

In [8]:
from langchain.memory import ConversationBufferMemory

# Crear una instancia de ConversationBufferMemory
memory = ConversationBufferMemory()

# Agregar mensajes de usuario y de IA a la memoria
memory.save_context({"input": "Hola"}, {"output": "Hola, ¿cómo estás?"})
memory.save_context({"input": "Muy bien, ¿y tú?"}, {"output": "Muy bien, gracias"})
memory.save_context({"input": "¿Qué haces?"}, {"output": "Estoy programando"})
memory.save_context({"input": "¿Qué lenguaje de programación?"}, {"output": "Python"})
memory.save_context({"input": "¿Qué es Python?"}, {"output": "Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código."})
# Leer los mensajes almacenados en la memoria
chat_history = memory.load_memory_variables({})
print(chat_history)

{'history': 'Human: Hola\nAI: Hola, ¿cómo estás?\nHuman: Muy bien, ¿y tú?\nAI: Muy bien, gracias\nHuman: ¿Qué haces?\nAI: Estoy programando\nHuman: ¿Qué lenguaje de programación?\nAI: Python\nHuman: ¿Qué es Python?\nAI: Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código.'}


## Integracion con el modelo

Para realizar esta integracion se usan los Chains

In [10]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

# Crear una instancia de OpenAI
llm = OpenAI(temperature=0)

# Crear una instancia de ConversationChain
conversation = ConversationChain(
    llm=llm,
    verbose=True, # True para imprimir los mensajes de la conversación
    memory=memory # Cargar la memoria de la conversación
)

In [11]:
# Iniciar una conversación 
conversation.predict(input="En que lenguaje Programas?") # input: mensaje de usuario se guarda en la memoria



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola
AI: Hola, ¿cómo estás?
Human: Muy bien, ¿y tú?
AI: Muy bien, gracias
Human: ¿Qué haces?
AI: Estoy programando
Human: ¿Qué lenguaje de programación?
AI: Python
Human: ¿Qué es Python?
AI: Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código.
Human: En que lenguaje Programas?
AI:

> Finished chain.


' Programo en Python.'

In [13]:
# Continuamos la convercación
conversation.predict(input="Cúal es tu profesión?") # input: mensaje de usuario se guarda en la memoria



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola
AI: Hola, ¿cómo estás?
Human: Muy bien, ¿y tú?
AI: Muy bien, gracias
Human: ¿Qué haces?
AI: Estoy programando
Human: ¿Qué lenguaje de programación?
AI: Python
Human: ¿Qué es Python?
AI: Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código.
Human: En que lenguaje Programas?
AI:  Programo en Python.
Human: Que haces?
AI:  Estoy programando una aplicación para ayudar a la gente a organizar sus tareas diarias.
Human: Cúal es tu profesión?
AI:

> Finished chain.


' Soy un programador de software.'

Como se puede ver gracias al verbose, la IA esta creando una memoria de la conversacion

# Conversation Knowledge Graph

## Creacion de la memoria

In [21]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI

# Crear una instancia de ConversationKGMemory
llm = OpenAI(temperature=0, verbose=True)
memory = ConversationKGMemory(llm=llm)

# Guardar el contexto de la conversación en la memoria
memory.save_context({"input": "Hola"}, {"output": "Hola, ¿cómo estás?"})
memory.save_context({"input": "Muy bien, ¿y tú?"}, {"output": "Muy bien, gracias"})
memory.save_context({"input": "¿Qué haces?"}, {"output": "Estoy programando"})
memory.save_context({"input": "¿Qué lenguaje de programación?"}, {"output": "Python"})
memory.save_context({"input": "¿Qué es Python?"}, {"output": "Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código."})
memory.save_context({"input": "Mi amigo pedro tambien es programador"}, {"output": "¿En Qué lenguaje de programación?"})
memory.save_context({"input": "En Java, Está especializado en el desarrollo de aplicaciones web y móviles."}, {"output": "Java es un lenguaje de programación de propósito general, concurrente, orientado a objetos que fue diseñado específicamente para tener tan pocas dependencias de implementación como fuera posible."})
# Cargar las variables de memoria
memory_variables = memory.load_memory_variables({"input": "Quien es pedro?"})
print(memory_variables)

{'history': 'On Pedro: Pedro is a programmer. Pedro is specialized in development of web and mobile applications. Pedro programs in Java.'}


## Integracion con el modelo

In [23]:
from langchain.chains import ConversationChain


# Crear una cadena de conversación con memoria
conversation_with_kg = ConversationChain(llm=llm, memory=memory, verbose=True)

# Predecir una respuesta utilizando la cadena de conversación
response = conversation_with_kg.predict(input="En que lenguaje programa Pedro?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
On Pedro: Pedro is a programmer. Pedro is specialized in development of web and mobile applications. Pedro programs in Java. Pedro is studying Software Engineering. Pedro is in second year. Pedro is at Harvard University.
Human: En que lenguaje programa Pedro?
AI:

> Finished chain.
 Pedro programa en Java.


Como podemos apreciar el modelo creo una relacion de grafos Con Pedro lo que le permite tener un contexto sobre que hace pedro y su especializacion